In [1]:
import yaml
import torch
import torch.onnx
import numpy as np
import backbone as bb
import onnx
import onnxruntime
import imp

In [2]:
imp.reload(bb)
project_name = "coco"
compound_coef = 0
input_size = [512, 640, 768, 896, 1024, 1280, 1280, 1536]
sample_batch_size = 1
device = torch.device("cpu")
onnx_name = "onnx_efficientdet_d%d.onnx" % compound_coef

In [3]:
params = yaml.safe_load(open(f'parameter/{project_name}.yml'))
obj_list = params['obj_list']    
weights_path = 'checkpoints/efficientdet-d%d.pth' % 0

model = bb.EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(obj_list),
                             ratios=eval(params['anchors_ratios']), 
                             scales=eval(params['anchors_scales']),
                             onnx_export=True).to(device)

model.backbone_net.model.set_swish(memory_efficient=False)
model.load_state_dict(torch.load(weights_path))  # , map_location=torch.device('cpu')))
model.requires_grad_(False)
model.eval()

EfficientDetBackbone(
  (regressboxes): BBoxTransform()
  (clipboxes): ClipBoxes()
  (bifpn): Sequential(
    (0): BiFPN(
      (conv6_up): SeparableConvBlock(
        (depthwise_conv): Conv2dStaticSamePadding(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), groups=64, bias=False)
        )
        (pointwise_conv): Conv2dStaticSamePadding(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        )
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
      (conv5_up): SeparableConvBlock(
        (depthwise_conv): Conv2dStaticSamePadding(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), groups=64, bias=False)
        )
        (pointwise_conv): Conv2dStaticSamePadding(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        )
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
      (conv4_up): SeparableConvBlo

In [4]:
dummy_input = torch.randn((sample_batch_size, 3, input_size[compound_coef], input_size[compound_coef]), 
                          dtype=torch.float32).to(device)

# opset_version can be changed to 10 or other number, based on your need
torch.onnx.export(model, dummy_input,
                  onnx_name,
                  verbose=False,
                  opset_version=10,
                  input_names = ['input'],  
                  output_names = ['output'])

/project/bo/efficient_traffic_count_on_edge_devices/efficientnet/utils_extra.py:34: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  h, w = x.cpu().detach().numpy().shape[-2:]
/project/bo/efficient_traffic_count_on_edge_devices/efficientnet/utils_extra.py:77: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  h, w = x.cpu().detach().numpy().shape[-2:]
/project/bo/efficient_traffic_count_on_edge_devices/efficientdet/utils.py:111: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python value

In [9]:
import onnx
model_onnx = onnx.load(onnx_name)
# Check that the IR is well formed
onnx.checker.check_model(model_onnx)
# Print a human readable representation of the graph


In [10]:
import onnxruntime
dummy_input = torch.randn((sample_batch_size, 3, input_size[compound_coef], input_size[compound_coef]), 
                          dtype=torch.float32, requires_grad=True).to(device)
ort_session = onnxruntime.InferenceSession(onnx_name)
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [11]:
###########################################################
# check the model output given an image, sucess           #
###########################################################

import utils.utils as input_utils
import vis_utils as vu
import time
img_path = "/project_scratch/bo/normal_data/aic2020/AIC20_track1/Dataset_A_Frame/cam_1_dawn/frame_00000304.jpg"
# img_path = "frames/clip/frame_00000020.jpg"
time_init = time.time()


ori_imgs, framed_imgs, \
    framed_metas = input_utils.preprocess(img_path, max_size=input_size[compound_coef])
x = torch.from_numpy(framed_imgs[0]).to(device).unsqueeze(0).permute(0, 3, 1, 2)

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)
regression, classification, anchors = ort_outs

threshold = 0.3
nms_threshold = 0.4
preds = input_utils.postprocess_npy(anchors, regression, classification, threshold, nms_threshold)
preds = input_utils.invert_affine_npy(framed_metas, preds)[0]
print("inference time", 1/(time.time() - time_init))

vu.show_bbox(ori_imgs[0], preds[:, :4], preds[:, 4], "pred", show=True)


ValueError: not enough values to unpack (expected 4, got 1)

In [5]:
sample_batch_size = 1
dummy_input = torch.randn((sample_batch_size, 3, input_size[compound_coef], input_size[compound_coef]))
traced_model = torch.jit.trace(model, dummy_input)

In [6]:
import coremltools as ct
model_ml = ct.convert(traced_model, inputs=[ct.ImageType(name="data", shape=dummy_input.shape)],)

Converting Frontend ==> MIL Ops:   0%|          | 10/7099 [00:00<00:08, 880.53 ops/s]


RuntimeError: PyTorch convert function for op 'constant_pad_nd' not implemented.

In [4]:
###########################################################
# convert the onnx model to coreml for iphone             #
###########################################################
import coremltools as ct
# from onnx_coreml import convert
import onnx

compound_coef = 0
onnx_name = "onnx_efficientdet_d%d.onnx" % compound_coef
model_onnx = onnx.load(onnx_name)
# model_file = open(onnx_name, 'rb')
# model_proto = onnx.onnx_pb.ModelProto()
# model_proto.ParseFromString(model_file.read())

In [7]:
coreml_model = convert(onnx_name, image_input_names=['input'], image_output_names=['output'], minimum_ios_deployment_target="13")

NotImplementedError: Unsupported ONNX ops of type: ATen 

In [10]:
model_coreml = convert(model_onnx, image_input_names=['input'], minimum_ios_deployment_target="13")

NotImplementedError: Unsupported ONNX ops of type: ATen 

In [5]:
import coremltools

model_ml = coremltools.converters.onnx.convert(model_onnx, image_input_names=["input"], minimum_ios_deployment_target="13")

NotImplementedError: Unsupported ONNX ops of type: ATen 

In [3]:
mlname = "coreml_efficientdet_d%d.mlmodel" % compound_coef
from onnx_coreml import convert
model_ml = convert(model=onnx_name, target_ios="12",)

ModuleNotFoundError: No module named 'coremltools.converters.nnssa'

In [ ]:

model_ml.save(mlname)

In [ ]:
ct.converters.onn